# b2210765024 İlbey GÜLMEZ AIN313 Assignment 2

Import necessary libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Part I 

## Maximum Likelihood Estimate (MLE)

### Question 1

In [2]:
coin_flips = ['H','T','H','T','H']

probability_heads = coin_flips.count('H') / len(coin_flips)

print("MLE for getting heads :", probability_heads)

MLE for getting heads : 0.6


### Question 2

In [3]:
candy_prices = np.array([2,3,5])
mean_candy_prices = np.mean(candy_prices)
print("MLE for the mean :", mean_candy_prices)

MLE for the mean : 3.3333333333333335


### Question 3

In [4]:
observations = [3,0,2,1,3,2,1,0,2,1]
# Since 0 <= θ <= 1, probability of occurences of 0 and 1 gives us the MLE for θ.
mle_theta = (observations.count(0) + observations.count(1)) / len(observations)
print("MLE for θ is", mle_theta)

MLE for θ is 0.5


### Question 4

In an uniform distribution, MLE of θ is the maximum observed value. So max value of X is the MLE of the sample.

## Naive Bayes Classifier

From the data:

P(Credit = Yes) = 5/10 = 0.5

Now, let's calculate the conditional probabilities for the attributes:

Sex:

P(Sex = Male | Credit = Yes) = 3/4 = 0.75

Education:

P(Education = University | Credit = Yes) = 2/4 = 0.5

Age:

P(Age = 16-25 | Credit = Yes) = 0/3 = 0

Let's apply Laplace Smoothing:

0 + 1 / 3 + number of attributes (6)
= 1/9

Income:

P(Income = Working Class | Credit = Yes) = 2/4 = 0.5

Now, let's calculate the probability for a 23-year-old male university graduate working class customer to get credit:

P(Credit = Yes | Male, University, 16-25, Working Class) = P(Credit = Yes) * P(Sex = Male | Credit = Yes) * P(Education = University | Credit = Yes) * P(Age = 16-25 | Credit = Yes) * P(Income = Working Class | Credit = Yes)

P(Credit = Yes | Male, University, 16-25, Working Class) = 0.5 * 0.75 * 0.5 * 1/9 * 0.5 = 0.0104

## Part II: Semantic Segmentation of Multispectral Images

### Load the dataset

In [5]:
classes = np.load('classes.npy')
train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')
train_masks = np.load('train_mask.npy')

In [6]:
print(*classes)

0. Other Class/Image Border 1. Road Markings, Asphalt, Landing Pad 2. Water 3. Building 4. Vehicle (Car, Truck, or Bus) 5. Person 6. Vegetation 7. Wood Panel 8. Rocks, Sand 9. Chair, Table


### Mask the data and labels

In [7]:
train_data = train_data[train_masks != 0]
train_labels = train_labels[train_masks != 0]

### Split the image into train and test splits

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    train_data,
    train_labels,
    test_size=0.2,
    random_state = 123
)

### Implement Naive Bayes Classification and accuracy functions

In [9]:
def calculate_mean_variance_prior(X_train, y_train):
    unique_labels = np.unique(y_train)
    parameters = {}
    for label in unique_labels:
        X_train_class = X_train[y_train == label]
        parameters[label] = {
            "mean": X_train_class.mean(axis = 0),
            "variance": X_train_class.var(axis = 0),
            "prior": X_train_class.shape[0] / X_train.shape[0]
        }
    return parameters

def gaussian_probability(X,mean,variance):
    exponent = -0.5 * np.sum(np.log(2. * np.pi * variance))
    exponent -= 0.5 * np.sum(((X - mean) ** 2) / (2 * variance), 1)
    return exponent
    
def naive_bayes_classification(X_test,y_train,parameters):
    predicted_labels = {}
    probabilities = np.zeros((X_test.shape[0], len(np.unique(y_train))))
    for class_, parameter in parameters.items():
        PX_y = gaussian_probability(X_test,parameter["mean"][0],parameter["variance"][0])
        probabilities[: , class_] = PX_y + np.log(parameter['prior'])
        predicted_labels = np.unique(y_train)[np.argmax(probabilities, axis=1)]
    return predicted_labels

def calculate_accuracy(predicted_labels,test_labels):
    return (predicted_labels == test_labels).sum() / len(test_labels)

def train_evaluate(X_train,X_test,y_train,y_test):
    parameters = calculate_mean_variance_prior(X_train, y_train)
    predicted_labels = naive_bayes_classification(X_test,y_train,parameters)
    return calculate_accuracy(predicted_labels, y_test)


### Calculate accuracy for RGB layers

In [10]:
rgb_accuracy = train_evaluate(X_train[:, :3], X_test[:, :3], y_train, y_test)

print(f"Accuracy for RGB layers is : {rgb_accuracy : .3f}")

Accuracy for RGB layers is :  0.611


### Calculate accuracy for Infrared layers

In [11]:
infrared_accuracy = train_evaluate(X_train[:, 3:], X_test[:, 3:],y_train,y_test)
print(f"Accuracy for Infrared layers is : {infrared_accuracy : .3f}")

Accuracy for Infrared layers is :  0.598


### Calculate accuracy for all layers

In [12]:
accuracy = train_evaluate(X_train,X_test,y_train,y_test)
print(f"Accuracy for all layers is : {accuracy : .3f}")

Accuracy for all layers is :  0.336


### Calculate accuracy for each layer

In [13]:
for index in range(X_train.shape[-1]):
    current_acc = train_evaluate(X_train[:, index : index + 1],
                                 X_test[:, index : index + 1],y_train,y_test)
    print(f"Accuracy for band {index + 1} : {current_acc : .3f}")

Accuracy for band 1 :  0.635
Accuracy for band 2 :  0.530
Accuracy for band 3 :  0.587
Accuracy for band 4 :  0.587
Accuracy for band 5 :  0.705
Accuracy for band 6 :  0.726


## Conclusion

In this assignment;

Demonstrated MLE in diverse scenarios, establishing a foundation for subsequent applications.

Successfully applied the classifier to individual spectral bands, RGB layers, Infrared layers, and the entire set of six spectral bands.

Attained accuracies of 61.1% for RGB layers, 59.8% for Infrared layers, and 33.6% for all six bands, revealing varied performance. This variety might be caused by numerous reasons such as unbalanced data. But the key conclusion is that the layers yield relatively good accuracies when evaluated seperately but yield poor score when combined.

Examined accuracy for each layer individually, highlighting distinctions in classification performance.

In summary, this assignment demonstrated MLE calculations and implemented a Gaussian Naive Bayes classifier for the semantic segmentation of multispectral images using the RIT-18 dataset.


#### Note:

I personally learned:

- How to load .npy files to python

- MLE calculations

- Semantic segmentation concept

- Masking a data

- Using log() on Gaussian Probability density function to prevent underflow and decrease runtime

- Implementing Naive Bayes Classifier manually in Python